In [1]:
#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

require("data.table")
require("scales")
require("lightgbm")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,613895,32.8,1330087,71.1,1171448,62.6
Vcells,1149448,8.8,8388608,64.0,1801055,13.8


Loading required package: data.table

Loading required package: scales

Loading required package: lightgbm



In [2]:
#Parametros del script
#setwd("C:/_MCD/Labo1/code")  #Establezco el Working Directory
setwd("~/buckets/b1/")

#cargo el dataset
dataset  <- fread("./datasets/c2_reparacEstad_fe_inicial_rank0fijo.csv.gz")   #donde entreno# FIN Parametros del script
nombre_base_pdf <- "densidades_reparacEstad_fe_inicial_rank0fijo"

In [3]:
kexperimento <- "comp2-EDA"
#creo la carpeta donde va el experimento
dir.create( paste0( "./exp/", kexperimento, "/"), showWarnings = FALSE )
setwd(paste0( "./exp/", kexperimento, "/"))   #Establezco el Working Directory DEL EXPERIMENTO


In [4]:
#creo la clase_binaria SI={ BAJA+1, BAJA+2 }    NO={ CONTINUA }
dataset[ foto_mes<=202103, 
         clase01 :=  ifelse( clase_ternaria=="CONTINUA", 0, 1 ) ]

dataset[  , foto_mes := as.character( foto_mes ) ]

In [5]:
campos_ordenados <- setdiff( colnames( dataset), 
                           c("numero_de_cliente","foto_mes","clase_ternaria" ) )


In [6]:
v_2019  <- c("201901","201902","201903","201904","201905","201906","201907","201908","201909","201910","201911","201912")
v_2020  <- c("202001","202002","202003","202004","202005","202006","202007","202008","202009","202010","202011","202012")
v_2021  <- c("202101", "202102", "202103","202104", "202105")

GLOBAL_colores <-  viridis_pal()(length(v_2019)+length(v_2020)+length(v_2021) )

In [7]:
graficar_campo  <- function( campo, periodos_analisis )
{

  #quito de grafico las colas del 5% de las densidades

  tb_quantiles  <- data.table( qmin= numeric(),  qmax= numeric() )

  for( periodo in periodos_analisis )
  {
    qu  <- quantile(  dataset[ foto_mes==periodo , get(campo) ] , prob= c(0.05, 0.95), na.rm=TRUE )
    tb_quantiles  <- rbind( tb_quantiles, use.names=FALSE, as.list(qu) )
  }

  xxmin  <- tb_quantiles[ , min( qmin ) ]
  xxmax  <- tb_quantiles[ , max( qmax ) ]

  yymax  <- 0
  for( per in periodos_analisis )
  {
    den  <- density( dataset[ foto_mes==per, get(campo) ],
                     kernel="gaussian", na.rm=TRUE )

    mayor  <- max( den$y )
    if( mayor > yymax ) yymax <- mayor 
  }

  densidad_A  <- density( dataset[ foto_mes==periodos_analisis[1], get(campo) ],
                          kernel="gaussian", na.rm=TRUE )


  plot( densidad_A,
        col= GLOBAL_colores[1],
        xlim= c( xxmin, xxmax ),
        ylim= c( 0, yymax ),
        main= paste0( campo  ) 
      )

  for( per in 2:length(periodos_analisis) )
  {
    densidad_B  <- density( dataset[ foto_mes==periodos_analisis[ per ], get(campo) ],
                            kernel="gaussian", na.rm=TRUE )

    lines(densidad_B, col= GLOBAL_colores[per], lty=1)
  }
  
  legend(  "topright",  
           legend= periodos_analisis,
           col=GLOBAL_colores, lty=1 )

}

In [8]:
pdf(paste0(nombre_base_pdf,"_todo.pdf"))

for( campo in  campos_ordenados )
{
  cat( campo, "  " )

  graficar_campo( campo, append(append(v_2019,v_2020),v_2021) )
}

dev.off()

active_quarter   cliente_vip   cliente_edad   cliente_antiguedad   cproductos   tcuentas   ccuenta_corriente   ccaja_ahorro   cdescubierto_preacordado   ctarjeta_debito   ctarjeta_debito_transacciones   ctarjeta_visa   ctarjeta_visa_transacciones   ctarjeta_master   ctarjeta_master_transacciones   cprestamos_personales   cprestamos_prendarios   cprestamos_hipotecarios   cplazo_fijo   cinversion1   cinversion2   cseguro_vida   cseguro_auto   cseguro_vivienda   cseguro_accidentes_personales   ccaja_seguridad   cpayroll_trx   cpayroll2_trx   ccuenta_debitos_automaticos   ctarjeta_visa_debitos_automaticos   ctarjeta_master_debitos_automaticos   cpagodeservicios   cpagomiscuentas   ccajeros_propios_descuentos   ctarjeta_visa_descuentos   ctarjeta_master_descuentos   ccomisiones_mantenimiento   ccomisiones_otras   cforex   cforex_buy   cforex_sell   ctransferencias_recibidas   ctransferencias_emitidas   cextraccion_autoservicio   ccheques_depositados   ccheques_emitidos   ccheques_depositado

ERROR: Error in density.default(dataset[foto_mes == per, get(campo)], kernel = "gaussian", : need at least 2 points to select a bandwidth automatically


In [ ]:
pdf(paste0(nombre_base_pdf,"_2020y2021.pdf")

for( campo in  campos_ordenados )
{
  cat( campo, "  " )

  graficar_campo( campo, append(v_2020,v_2021) )
}

dev.off()

In [ ]:
pdf(paste0(nombre_base_pdf,"_2021.pdf")

for( campo in  campos_ordenados )
{
  cat( campo, "  " )

  graficar_campo( campo, v_2021 )
}

dev.off()

In [ ]:
pdf(paste0(nombre_base_pdf,"_desde202007.pdf")

for( campo in  campos_ordenados )
{
  cat( campo, "  " )

  graficar_campo( campo, append(c("202007","202008","202009","202010","202011","202012"),v_2021) )
}

dev.off()